# MINE API Examples
MINE is a database of predicted metabolic reactions, based on known biochemistry. You can use the MINE API to:

1. Perform a database search, including MINE compound search by similarity or substructure
2. Retrieve lists of compounds, operators, and reactions
3. Perform an MS search (requires MS data)
4. Perform an MS2 search (requires MS2 data)
5. Download predicted spectra for MINE compounds
6. Get a list of possible adducts for metabolomics searches

Examples are provided below for each of these requests. The mongotest MINE database, which contains 26 compounds, 8 reactions, and 2 operators, is used for all examples.

**Note:** If using python requests library, make sure verify argument to get() or post() method is set to False. We are seeking to fix this in the future.

## Setup

In [1]:
import requests
import json

In [2]:
BASE_URL = 'https://minedatabase.ci.northwestern.edu/mineserver/'

## 1. Perform a database search
All searches are GET requests.

**Endpoints**:

- **Quick Search**: .../mineserver/quick-search/&lt;db_name>/q=&lt;query>

- **Similarity Search**: ...mineserver/similarity-search/&lt;db_name>/smiles=&lt;smiles>/&lt;min_tc>/&lt;limit>  
....where &lt;min_tc> and &lt;limit> are optional.

- **Substructure Search**: .../mineserver/substructure-search/&lt;db_name>/smiles=&lt;smiles>/&lt;limit>  
....where &lt;limit> is optional.

- **Model Search**: .../mineserver/model-search/q=&lt;query>

- **Database Search**: .../mineserver/database-query/&lt;db_name>/q=&lt;mongo_query>

**URL Arguments**:
- &lt;db_name>: Name of MINE database
- &lt;query>: Search query - format depends on the search:
 - Quick Search: a MINE id, KEGG organism code, ModelSEED id, Inchikey, or Name
 - Model Search: KEGG organism code or organism name
 - Database Search: a valid Mongo query
- &lt;smiles>: SMILES string describing molecular structure (or substructure for substructure search)
- &lt;min_tc>: (optional) minimum Tanimoto coefficient for similarity scoring
- &lt;limit>: (optional) maximum number of results to return

Quick search, similarity search, and substructure search will likely be the most common use cases. Model search can be used to see whether MINE has compounds linked to a specific organism. Database search can be used for completely customizable searches as it just requires a valid search query using Mongo syntax.

### Example 1a - Quick Search
Here, we perform a quick search based on a compound name

In [10]:
url = BASE_URL + 'quick-search/mongotest/q=4-aminobutanal'
import certifi
response = requests.get(url, verify=False)
json.loads(response.content)

C:\Users\jonst\Anaconda3\envs\MINE\lib\site-packages\urllib3\connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minedatabase.ci.northwestern.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


[{'DB_links': {'3DMET': ['B00129'],
   'ChEBI': ['17769', '27390'],
   'KEGG': ['C02903', 'C00555'],
   'KNApSAcK': ['C00019656'],
   'Model_SEED': ['cpd12219', 'cpd00434'],
   'NIKKAJI': ['J38.531J'],
   'PubChem': ['5830', '3835']},
  'Formula': 'C4H9NO',
  'Generation': 0,
  'Inchikey': 'DZQLQEYLEYWJIB-UHFFFAOYSA-N',
  'MINE_id': 13565,
  'Mass': 87.068413908,
  'NP_likeness': 1.953252287376267,
  'Names': ['4-Aminobutanal', 'omega-Aminoaldehyde', '4-Aminobutyraldehyde'],
  'SMILES': 'NCCCC=O',
  '_id': 'C32389afd9400899c48c81ea461fb97db215dc733'}]

### Example 1b - Similarity Search
Here, we perform a similarity search using D-glucose as the query molecule. D-glucose is in the database so it is returned. D-allose is also returned.

In [14]:
url = BASE_URL + 'similarity-search/mongotest/smiles=C(C1C(C(C(C(O1)O)O)O)O)O'
response = requests.get(url, verify=False)
json.loads(response.content)

C:\Users\jonst\Anaconda3\envs\MINE\lib\site-packages\urllib3\connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minedatabase.ci.northwestern.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


[{'DB_links': {'3DMET': ['B04623'],
   'CAS': ['50-99-7', '2280-44-6'],
   'ChEBI': ['4167'],
   'KEGG': ['C00031'],
   'KNApSAcK': ['C00042470'],
   'Model_SEED': ['cpd26821', 'cpd00027'],
   'NIKKAJI': ['J4.109B'],
   'PDB-CCD': ['BGC', 'GLC'],
   'PubChem': ['3333']},
  'Formula': 'C6H12O6',
  'Generation': 1,
  'Inchikey': 'WQZGKKKJIJFFOK-GASJEMHNSA-N',
  'MINE_id': 917030,
  'Mass': 180.063388104,
  'NP_likeness': 2.62691337083175,
  'Names': ['Glucose',
   'Dextrose',
   'D-Glucose',
   'Grape sugar',
   'D-Glucopyranose'],
  'SMILES': 'OC[C@H]1OC(O)[C@H](O)[C@@H](O)[C@@H]1O',
  'Sources': [{'Compounds': ['C08a914cde05039694ef0194d9ee79ff9a79dde33',
     'C2ce917f1d3aaef7501595123894a68a7d786a9e7'],
    'Operators': ['3.2.1.a']},
   {'Compounds': ['C2c3b118fb2dbd237cc8a4878b0643385860fb427',
     'C08a914cde05039694ef0194d9ee79ff9a79dde33'],
    'Operators': ['3.2.1.a']},
   {'Compounds': ['C08a914cde05039694ef0194d9ee79ff9a79dde33',
     'Ca67dd0794223d284a9517566c3fd4107728e080

### Example 1c - Substructure Search
Here, we perform a substructure search using a part of the D-glucose molecule (just a C-C-C-O structure)

In [15]:
url = BASE_URL + 'substructure-search/mongotest/smiles=CCCO'
response = requests.get(url, verify=False)
json.loads(response.content)

C:\Users\jonst\Anaconda3\envs\MINE\lib\site-packages\urllib3\connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minedatabase.ci.northwestern.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


[{'DB_links': {'3DMET': ['B04623'],
   'CAS': ['50-99-7', '2280-44-6'],
   'ChEBI': ['4167'],
   'KEGG': ['C00031'],
   'KNApSAcK': ['C00042470'],
   'Model_SEED': ['cpd26821', 'cpd00027'],
   'NIKKAJI': ['J4.109B'],
   'PDB-CCD': ['BGC', 'GLC'],
   'PubChem': ['3333']},
  'Formula': 'C6H12O6',
  'Generation': 1,
  'Inchikey': 'WQZGKKKJIJFFOK-GASJEMHNSA-N',
  'MINE_id': 917030,
  'Mass': 180.063388104,
  'NP_likeness': 2.62691337083175,
  'Names': ['Glucose',
   'Dextrose',
   'D-Glucose',
   'Grape sugar',
   'D-Glucopyranose'],
  'SMILES': 'OC[C@H]1OC(O)[C@H](O)[C@@H](O)[C@@H]1O',
  'Sources': [{'Compounds': ['C08a914cde05039694ef0194d9ee79ff9a79dde33',
     'C2ce917f1d3aaef7501595123894a68a7d786a9e7'],
    'Operators': ['3.2.1.a']},
   {'Compounds': ['C2c3b118fb2dbd237cc8a4878b0643385860fb427',
     'C08a914cde05039694ef0194d9ee79ff9a79dde33'],
    'Operators': ['3.2.1.a']},
   {'Compounds': ['C08a914cde05039694ef0194d9ee79ff9a79dde33',
     'Ca67dd0794223d284a9517566c3fd4107728e080

### Example 1d - Model Search
Here, we see if an organism code exists in the MINE database.

In [11]:
url = BASE_URL + 'model-search/q=eco'  # E. coli
response = requests.get(url, verify=False)
json.loads(response.content)

C:\Users\jonst\Anaconda3\envs\MINE\lib\site-packages\urllib3\connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minedatabase.ci.northwestern.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


['eco', 'ecos']

### Example 1e - Database Search
Here, we search for structures matching the formula, "C6H13O9P" using a mongo query. Any field in the returned result (e.g. 'DB_links', 'Formula', 'Generation', etc.) can be queried.

In [18]:
url = BASE_URL + 'database-query/mongotest/q={"Formula": "C6H13O9P"}'
response = requests.get(url, verify=False)
json.loads(response.content)

C:\Users\jonst\Anaconda3\envs\MINE\lib\site-packages\urllib3\connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minedatabase.ci.northwestern.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


[{'DB_links': {'3DMET': ['B01244'],
   'CAS': ['2255-14-3'],
   'ChEBI': ['17973'],
   'KEGG': ['C00446'],
   'KNApSAcK': ['C00007391'],
   'Model_SEED': ['cpd00348'],
   'NIKKAJI': ['J92.787B'],
   'PDB-CCD': ['GL1'],
   'PubChem': ['3734']},
  'Formula': 'C6H13O9P',
  'Generation': 0,
  'Inchikey': 'HXXFSFRBOHSIMQ-FPRJBGLDSA-N',
  'MINE_id': 1001725,
  'Mass': 260.02971862600003,
  'NP_likeness': 2.150848501723131,
  'Names': ['alpha-D-Galactose 1-phosphate',
   'alpha-D-Galactopyranose 1-phosphate'],
  'SMILES': 'O=P(O)(O)O[C@H]1O[C@H](CO)[C@H](O)[C@H](O)[C@H]1O',
  '_id': 'C03e0b10e6490ce79a7b88cb0c4e17c2bf6204352'}]

## 2. Retrieve Lists of Compounds, Reactions, and Operators
Compounds, Reactions, and Operators are each a collection within each MINE database. get-ids is a GET request while get-comps, get-rxns, and get-ops are POST requests. get-op-w-rxns is a GET request.

**Endpoints**:
- **Get IDs**: .../mineserver/get-ids/&lt;db_name>/&lt;collection_name>/q=&lt;query>  
  ....where q=&lt;query> is optional. 
- **Get Compounds**: .../mineserver/get-comps/&lt;db_name>
- **Get Reactions**: .../mineserver/get-rxns/&lt;db_name>
- **Get Operators**: .../mineserver/get-ops/&lt;db_name>
- **Get Operator with Reactions**: .../mineserver/get-op-w-rxns/&lt;db_name>/&lt;op_id>

**URL Arguments**:
- &lt;db_name>: name of MINE database
- &lt;collection_name>: name of collection with in MINE database
- &lt;query>: get only IDs for documents matching this query (in Mongo syntax)
- &lt;op_id>: operator ID (e.g. 1.1.-1.h) or Mongo ID for operator document

**JSON Arguments (for POST requests)**:
- id_list: List of IDs. Can be either MINE IDs or Mongo document IDs.
 - can specify for get-comps, get-rxns, or get-ops POST requests

A general workflow to retrieve all the compound information for a MINE database could be to (1) get all IDs for compounds in the compound collection (via get-ids), and to then (2) get info on all those compounds via get-comps.

### Example 2a - Get IDs
Here, we retrieve IDs of all compounds, reactions, and operators in the mongotest database. For compounds and reactions, this is the Mongo document ID. For operators, this is the operator name (e.g. 2.7.1.a).

#### For Compounds

In [25]:
url = BASE_URL + 'get-ids/mongotest/compounds'
response = requests.get(url, verify=False)
json.loads(response.content)

C:\Users\jonst\Anaconda3\envs\MINE\lib\site-packages\urllib3\connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minedatabase.ci.northwestern.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


['Ccffda1b2e82fcdb0e1e710cad4d5f70df7a5d74f',
 'C03e0b10e6490ce79a7b88cb0c4e17c2bf6204352',
 'C189c6f78587772ccbd0c2b9e118cf88ff35c316a',
 'C1b4c5a4d3fadf6ca4e9d359047e5fa17f2c30110',
 'C32389afd9400899c48c81ea461fb97db215dc733',
 'C42e329d8ad416dc34cbccf0a367b09122fc30b56',
 'C455bc3dc93cd3bb3bef92a34767693a4716aa3fb',
 'C4c3782cb869285f550e1eca3afe7c436f157a1c3',
 'C5845b7a617e36c1b16e9f7dcab280a310c8823e5',
 'C5969e8ac0bddac35cd9fe7dc3cdd403a8ebf3a72',
 'C5f385767a986eb8bd689ff65fcb183570ada9cd1',
 'C6417809e439584834e40e8c8968c52adc20adb86',
 'C65bbbc584d68c8365c11ee9b1883c7876bf7820d',
 'C782d4dc99139098ea4eedc6c25e09609365f6f07',
 'C7e33140717566784c6d1afcfcca02a6d5800d25c',
 'C8fdd00cc70e0b9e57aa6f517c219b6b92284171a',
 'C90ba1f8d4d84f6305539f6d05a74497d4d5dfe06',
 'C920d7760804a8487a9345e93256fffe33e9efa55',
 'C9c69cbeb40f083118c1913599c12c7f4e5e68d03',
 'Ca9d4a12388fae3add26d70d3e8e523631c36ee75',
 'Cc5a77e982a276f02403c86b2f879302a806a7cc7',
 'Cc658bd993126bf4f1005a4fdcb25cd8

#### For Reactions

In [26]:
url = BASE_URL + 'get-ids/mongotest/reactions'
response = requests.get(url, verify=False)
json.loads(response.content)

C:\Users\jonst\Anaconda3\envs\MINE\lib\site-packages\urllib3\connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minedatabase.ci.northwestern.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


['15a593f15d00f1b62536229d50b425e931310c08cbdd6c49c71fa5f05da3ef10',
 '3753071b9e61b4dea1494f5373eda4a7d159f5b42d7e85a3b1017a8e1e3fb687',
 '8bec4e7a85c8a8cac20686178c4a91158c40906c034a4ef5c9a912d30fa80ef9',
 'b8cf960fdf4cb9299edd553654921379fd7662e1fafda6a52dc2ec29dc63da0a',
 'c5f98ea336f41db5587bdbbe275c48a620fd443d8a4fbbb3d000e7061e0e2430',
 'ea85b4626f9e3ded5da8c6b77523283b78ebd2d09fc11ff26beeb59842324e9f',
 'f70ecb8b0c93969e47d060278d2be979807d5224ed8d2bffc51dedeec9832884',
 '4542c96f4bca04bfe2db15bc71e9eaee38bee5b87ad8a6752a5c4718ba1974c1']

#### For Operators

In [27]:
url = BASE_URL + 'get-ids/mongotest/operators'
response = requests.get(url, verify=False)
json.loads(response.content)

C:\Users\jonst\Anaconda3\envs\MINE\lib\site-packages\urllib3\connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minedatabase.ci.northwestern.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


['2.7.1.a', 'test_op']

### Example 2b - Get Compounds
Here, we get info for the first two compounds returned from get-ids for the compound collection.

In [53]:
url = BASE_URL + 'get-comps/mongotest'
data = {'id_list': ['Ccffda1b2e82fcdb0e1e710cad4d5f70df7a5d74f',
                    'C03e0b10e6490ce79a7b88cb0c4e17c2bf6204352']}
response = requests.post(url, json=data, verify=False)
json.loads(response.content)

C:\Users\jonst\Anaconda3\envs\MINE\lib\site-packages\urllib3\connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minedatabase.ci.northwestern.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


[{'Charge': 0,
  'DB_links': {'3DMET': ['B04623'],
   'CAS': ['50-99-7', '2280-44-6'],
   'ChEBI': ['4167'],
   'KEGG': ['C00031'],
   'KNApSAcK': ['C00042470'],
   'Model_SEED': ['cpd26821', 'cpd00027'],
   'NIKKAJI': ['J4.109B'],
   'PDB-CCD': ['BGC', 'GLC'],
   'PubChem': ['3333']},
  'Enzymes': ['',
   '1.1.5.2',
   '2.4.1.9',
   '2.3.1.-',
   '3.1.3.9',
   '3.1.6.3',
   '2.7.1.1',
   '5.3.1.5',
   '2.4.1.8',
   '2.7.1.2',
   '1.1.5.9',
   '3.2.1.-',
   '3.1.3.58',
   '3.2.1.10',
   '3.2.1.22',
   '2.7.1.61',
   '3.2.1.20',
   '2.4.1.10',
   '1.1.3.10',
   '2.3.1.92',
   '3.2.1.62',
   '2.4.1.99',
   '3.2.1.48',
   '3.2.1.86',
   '2.4.1.22',
   '2.4.1.20',
   '3.2.1.21',
   '3.2.1.45',
   '2.7.1.42',
   '2.7.1.63',
   '3.1.1.33',
   '2.3.1.91',
   '3.2.1.28',
   '3.2.1.74',
   '3.1.4.48',
   '3.2.1.84',
   '2.4.1.64',
   '2.7.1.62',
   '3.2.1.93',
   '3.2.1.23',
   '2.3.1.90',
   '2.4.1.25',
   '3.1.3.10',
   '2.7.1.69',
   '3.2.1.39',
   '3.2.1.42',
   '2.3.1.72',
   '2.4.1.31',
 

### Example 2c - Get Reactions
Here, we get info for the first two reactions from get-ids for the reaction collection.

In [55]:
url = BASE_URL + 'get-rxns/mongotest'
data = {'id_list': ['15a593f15d00f1b62536229d50b425e931310c08cbdd6c49c71fa5f05da3ef10',
                    '3753071b9e61b4dea1494f5373eda4a7d159f5b42d7e85a3b1017a8e1e3fb687']}
response = requests.post(url, json=data, verify=False)
json.loads(response.content)

C:\Users\jonst\Anaconda3\envs\MINE\lib\site-packages\urllib3\connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minedatabase.ci.northwestern.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


[{'Generation': 0,
  'ID': 'pk_rxn0000001',
  'ID_rxn': '(1) pk_cpd0000002[c0] + (1) ATP[c0] => (1) pk_cpd0000004[c0] + (1) ADP[c0]',
  'Operators': ['2.7.1.a'],
  'Products': [{'c_id': 'C42e329d8ad416dc34cbccf0a367b09122fc30b56',
    'stoich': 1},
   {'c_id': 'C189c6f78587772ccbd0c2b9e118cf88ff35c316a', 'stoich': 1}],
  'Reactants': [{'c_id': 'Cd491500039dd5f38a72c9240b0655e6c919cb9e3',
    'stoich': 1},
   {'c_id': 'C9c69cbeb40f083118c1913599c12c7f4e5e68d03', 'stoich': 1}],
  'SMILES_rxn': '(1) Cc1cc2c(cc1C)N(CC(O)C(O)C(O)COP(=O)(O)OP(=O)(O)OCC1OC(n3cnc4c(N)ncnc43)C(O)C1O)c1nc(O)nc(O)c1N2 + (1) Nc1ncnc2c1ncn2[C@@H]1O[C@H](COP(=O)(O)OP(=O)(O)OP(=O)(O)O)[C@@H](O)[C@H]1O => (1) Cc1cc2c(cc1C)N(CC(O)C(O)C(O)COP(=O)(O)OP(=O)(O)OCC1OC(n3cnc4c(N)ncnc43)C(O)C1O)c1nc(OP(=O)(O)O)nc(O)c1N2 + (1) Nc1ncnc2c1ncn2[C@@H]1O[C@H](COP(=O)(O)OP(=O)(O)O)[C@@H](O)[C@H]1O',
  '_id': '15a593f15d00f1b62536229d50b425e931310c08cbdd6c49c71fa5f05da3ef10'},
 {'Generation': 0,
  'ID': 'pk_rxn0000002',
  'ID_rxn': '

### Example 2d - Get Operators
Here, we get info for the two operators from get-ids for the operator colleciton.

In [56]:
url = BASE_URL + 'get-ops/mongotest'
data = {'id_list': ['2.7.1.a', 'test_op']}
response = requests.post(url, json=data, verify=False)
json.loads(response.content)

C:\Users\jonst\Anaconda3\envs\MINE\lib\site-packages\urllib3\connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minedatabase.ci.northwestern.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


[{'Comments': 'Comment here',
  'Name': '2.7.1.a',
  'Products': ['cpd00008', 'Any'],
  'Reactants': ['cpd00002', 'Any'],
  'Reactions_predicted': 8,
  'SMARTS': '[#1;D1R0:1][#6:2][#6;H0D3R0:3](=[#8;H0D1R0:4])[#6;H0D3R0:5]([#8;H1D2R0:7])=[#8;H0D1R0:6].[#8;H1D2R0:11][#15;H0D4R0:8]([#8;H1D2R0:12])[#8;H0D2:9][#15;H0D4:10]>>[*:1]-[*:9]-[*:10].[*:11]-[*:8](-[*:12])-[*:4]-[*:3](=[*:2])-[*:5](-[*:7])=[*:6]',
  '_id': '2.7.1.a'},
 {'Comments': 'Comment here for test operator',
  'Name': 'test operator',
  'Products': ['Any'],
  'Reactants': ['Any'],
  'Reactions_predicted': 69,
  'SMARTS': '[#1;D1R0:1][#6:2][#6;H0D3R0:3](=[#8;H0D1R0:4])[#6;H0D3R0:5]([#8;H1D2R0:7])=[#8;H0D1R0:6].[#8;H1D2R0:11][#15;H0D4R0:8]([#8;H1D2R0:12])[#8;H0D2:9][#15;H0D4:10]>>[*:1]-[*:9]-[*:10].[*:11]-[*:8](-[*:12])-[*:4]-[*:3](=[*:2])-[*:5](-[*:7])=[*:6]',
  '_id': 'test_op'}]

### Example 2e - Get Reactions for Operator
Here, we get all reactions linked to a specific operator. Similar to get-ops but in this case we also get the reaction IDs.

In [57]:
url = BASE_URL + 'get-op-w-rxns/mongotest/2.7.1.a'
response = requests.get(url, verify=False)
json.loads(response.content)

C:\Users\jonst\Anaconda3\envs\MINE\lib\site-packages\urllib3\connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minedatabase.ci.northwestern.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


{'Comments': 'Comment here',
 'Name': '2.7.1.a',
 'Products': ['cpd00008', 'Any'],
 'Reactants': ['cpd00002', 'Any'],
 'Reaction_ids': ['15a593f15d00f1b62536229d50b425e931310c08cbdd6c49c71fa5f05da3ef10',
  '3753071b9e61b4dea1494f5373eda4a7d159f5b42d7e85a3b1017a8e1e3fb687',
  '8bec4e7a85c8a8cac20686178c4a91158c40906c034a4ef5c9a912d30fa80ef9',
  'b8cf960fdf4cb9299edd553654921379fd7662e1fafda6a52dc2ec29dc63da0a',
  'c5f98ea336f41db5587bdbbe275c48a620fd443d8a4fbbb3d000e7061e0e2430',
  'ea85b4626f9e3ded5da8c6b77523283b78ebd2d09fc11ff26beeb59842324e9f',
  'f70ecb8b0c93969e47d060278d2be979807d5224ed8d2bffc51dedeec9832884'],
 'Reactions_predicted': 8,
 'SMARTS': '[#1;D1R0:1][#6:2][#6;H0D3R0:3](=[#8;H0D1R0:4])[#6;H0D3R0:5]([#8;H1D2R0:7])=[#8;H0D1R0:6].[#8;H1D2R0:11][#15;H0D4R0:8]([#8;H1D2R0:12])[#8;H0D2:9][#15;H0D4:10]>>[*:1]-[*:9]-[*:10].[*:11]-[*:8](-[*:12])-[*:4]-[*:3](=[*:2])-[*:5](-[*:7])=[*:6]',
 '_id': '2.7.1.a'}

## 3. Perform an MS Search
Provide a monoisotopic mass as well as a dictionary of search parameters to search the MINE for matching compounds. This is a POST request. Here we just provide monoisotopic mass as regular text. However, you can also copy and paste mgf- or mzxml-formatted data in the 'text' field (and set 'text_type' to the appropriate value).

**Endpoint**: .../mineserver/ms-adduct-search/&lt;db_name>

**URL Arguments**:
- &lt;db_name>: name of MINE database

**JSON Arguments**:
- See API documentation.

### Example 3a - MS Search

In [58]:
url = BASE_URL + 'ms-adduct-search/mongotest'
json_dict = {
    'tolerance': 10,  # required (10 mDa)
    'charge': False,  # required (False = negative)
    'text': '259.02244262600003',  # monoisotopic mass (including adduct) goes here
    # rest of parameters are optional
    'text_type': 'form',
    'adducts': "['[M-H]-']",
    'models': "['eco']",
    'ppm': False,
    'halogens': True,
    'verbose': True
}
response = requests.post(url, json=json_dict, verify=False)
json.loads(response.content)

C:\Users\jonst\Anaconda3\envs\MINE\lib\site-packages\urllib3\connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minedatabase.ci.northwestern.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


[{'Formula': 'C6H13O9P',
  'Generation': 0,
  'Inchikey': 'HXXFSFRBOHSIMQ-FPRJBGLDSA-N',
  'MINE_id': 1001725,
  'NP_likeness': 2.150848501723131,
  'Names': ['alpha-D-Galactose 1-phosphate',
   'alpha-D-Galactopyranose 1-phosphate'],
  'Pos_CFM_spectra': {'10 V': [[73.02950298, 1.226207543],
    [78.95905407, 19.50435213],
    [96.96961875, 14.49386242],
    [103.0400677, 2.978890742],
    [133.0506323, 5.70349883],
    [138.9801834, 1.685878594],
    [229.0118775, 1.31830202],
    [241.0118775, 5.648170593],
    [259.0224422, 28.4906794]],
   '20 V': [[43.0189383, 1.519916692],
    [59.01385292, 1.137269632],
    [62.96413945, 1.679096142],
    [73.02950298, 1.507196829],
    [78.95905407, 49.95703371],
    [80.97470413, 1.297807233],
    [89.0244176, 1.254541085],
    [96.96961875, 11.87596697],
    [161.045547, 1.441077965],
    [179.0561117, 1.29782661],
    [241.0118775, 3.540979881],
    [259.0224422, 3.932687093]],
   '40 V': [[43.0189383, 0.8990633655],
    [59.01385292, 1.694

## 4. Perform an MS2 Search
Provide a list of MS2 spectra and intensities as well as a dictionary of search parameters to search the MINE for matching compounds. This is a POST request. Here we just provide monoisotopic mass as regular text. However, you can also copy and paste mgf- or mzxml-formatted data in the 'text' field (and set 'text_type' to the appropriate value). Spectral score indicates the quality of the match (from 0 to 1000).

**Endpoint**: .../mineserver/ms2-search/&lt;db_name>

**URL Arguments**:
- &lt;db_name>: name of MINE database

**JSON Arguments**:
- See API documentation.

### Example 4a - MS2 Search

In [59]:
url = BASE_URL + 'ms2-search/mongotest'
json_dict = {
    'tolerance': 10,  # in mDa
    'charge': True,  # True=positive
    'energy_level': 20,  # in volts
    'scoring_function': 'dot product',
    'text': '261.037\n43.0189 1\n59.013 1\n96.970 10',  # MS2 spectra go here
    # All following parameters are optional
    'text_type': 'form',
    'adducts': "['[M+H]+']",
    'models': "['hsa']",
    'ppm': False,
    'halogens': True,
    'verbose': True
}
response = requests.post(url, json=json_dict, verify=False)
json.loads(response.content)

C:\Users\jonst\Anaconda3\envs\MINE\lib\site-packages\urllib3\connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minedatabase.ci.northwestern.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


[{'Formula': 'C6H13O9P',
  'Generation': 0,
  'Inchikey': 'HXXFSFRBOHSIMQ-FPRJBGLDSA-N',
  'MINE_id': 1001725,
  'NP_likeness': 2.150848501723131,
  'Names': ['alpha-D-Galactose 1-phosphate',
   'alpha-D-Galactopyranose 1-phosphate'],
  'SMILES': 'O=P(O)(O)O[C@H]1O[C@H](CO)[C@H](O)[C@H](O)[C@H]1O',
  'Spectral_score': 232,
  '_id': 'C03e0b10e6490ce79a7b88cb0c4e17c2bf6204352',
  'adduct': '[M+H]+',
  'native_hit': True,
  'peak_name': '261.037'}]

## 5. Download Predicted Spectra
In the MINE database, spectra are predicted for every compound using CFM-ID. You can retrieve these predictions by using this endpoint. This is a POST request. Note that JSON is not returned. You have to decode the bytestring yourself.

**Endpoint**: .../mineserver/spectra-download/&lt;db_name>/q=&lt;mongo_query>  
where q=&lt;mongo_query> is optional

**URL Arguments**:
- &lt;db_name>: name of MINE database
- &lt;mongo_query>: query to filter results by (using Mongo syntax)

**JSON Arguments**:
- parent_filter: Filter by KEGG organism code
- putative: If True, also get spectra for predicted compounds

### Example 5a - Download all Spectra

In [84]:
url = 'http://127.0.0.1:5000/mineserver/' + 'spectra-download/mongotest'
response = requests.get(url, verify=False)
response.content.decode().split('\n')

['Name: alpha-D-Galactose 1-phosphate',
 'Synonym: alpha-D-Galactopyranose 1-phosphate',
 '_id: C03e0b10e6490ce79a7b88cb0c4e17c2bf6204352',
 'Formula: C6H13O9P',
 'Generation: 0',
 'Inchikey: HXXFSFRBOHSIMQ-FPRJBGLDSA-N',
 'MINE_id: 1001725',
 'Mass: 260.02971862600003',
 'SMILES: O=P(O)(O)O[C@H]1O[C@H](CO)[C@H](O)[C@H](O)[C@H]1O',
 'Instrument: CFM-ID',
 'Ionization: Positive',
 'Energy: 10 V',
 'Num Peaks: 9',
 '73.02950298 1.226207543',
 '78.95905407 19.50435213',
 '96.96961875 14.49386242',
 '103.0400677 2.978890742',
 '133.0506323 5.70349883',
 '138.9801834 1.685878594',
 '229.0118775 1.31830202',
 '241.0118775 5.648170593',
 '259.0224422 28.4906794',
 '',
 'Name: alpha-D-Galactose 1-phosphate',
 'Synonym: alpha-D-Galactopyranose 1-phosphate',
 '_id: C03e0b10e6490ce79a7b88cb0c4e17c2bf6204352',
 'Formula: C6H13O9P',
 'Generation: 0',
 'Inchikey: HXXFSFRBOHSIMQ-FPRJBGLDSA-N',
 'MINE_id: 1001725',
 'Mass: 260.02971862600003',
 'SMILES: O=P(O)(O)O[C@H]1O[C@H](CO)[C@H](O)[C@H](O)[C@H]1

### Example 5b - Download a Spectrum for a Specific Compound

In [115]:
url = BASE_URL + 'spectra-download/mongotest/q={"_id": "C03e0b10e6490ce79a7b88cb0c4e17c2bf6204352"}'
data = {"ID": "Ccffda1b2e82fcdb0e1e710cad4d5f70df7a5d74f"}
response = requests.post(url, json=data, verify=False)
response.content.decode().split('\n')

C:\Users\jonst\Anaconda3\envs\MINE\lib\site-packages\urllib3\connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minedatabase.ci.northwestern.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


['Name: alpha-D-Galactose 1-phosphate',
 'Synonym: alpha-D-Galactopyranose 1-phosphate',
 '_id: C03e0b10e6490ce79a7b88cb0c4e17c2bf6204352',
 'Formula: C6H13O9P',
 'Generation: 0',
 'Inchikey: HXXFSFRBOHSIMQ-FPRJBGLDSA-N',
 'MINE_id: 1001725',
 'Mass: 260.02971862600003',
 'SMILES: O=P(O)(O)O[C@H]1O[C@H](CO)[C@H](O)[C@H](O)[C@H]1O',
 'Instrument: CFM-ID',
 'Ionization: Positive',
 'Energy: 10 V',
 'Num Peaks: 9',
 '73.02950298 1.226207543',
 '78.95905407 19.50435213',
 '96.96961875 14.49386242',
 '103.0400677 2.978890742',
 '133.0506323 5.70349883',
 '138.9801834 1.685878594',
 '229.0118775 1.31830202',
 '241.0118775 5.648170593',
 '259.0224422 28.4906794',
 '',
 'Name: alpha-D-Galactose 1-phosphate',
 'Synonym: alpha-D-Galactopyranose 1-phosphate',
 '_id: C03e0b10e6490ce79a7b88cb0c4e17c2bf6204352',
 'Formula: C6H13O9P',
 'Generation: 0',
 'Inchikey: HXXFSFRBOHSIMQ-FPRJBGLDSA-N',
 'MINE_id: 1001725',
 'Mass: 260.02971862600003',
 'SMILES: O=P(O)(O)O[C@H]1O[C@H](CO)[C@H](O)[C@H](O)[C@H]1

## 6. Get List of Possible Metabolomics Adducts
Get names of adducts able to be used for metabolomics (MS and MS2) searches. This is a GET request.

**Endpoint**: .../mineserver/get-adduct-names/&lt;adduct_type>
where &lt;adduct_type> is optional

**URL Arguments**:
- &lt;adduct_type>: Type of adduct. Options are 'positive', 'negative', or 'all'. Defaults to 'all'.

### Example 6a - Get all Adduct Names

In [116]:
url = BASE_URL + 'get-adduct-names'
response = requests.get(url, verify=False)
json.loads(response.content)

C:\Users\jonst\Anaconda3\envs\MINE\lib\site-packages\urllib3\connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minedatabase.ci.northwestern.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


[['[M]+',
  '[M+H]+',
  '[M+NH4]+',
  '[M+Na]+',
  '[M+2H]+',
  '[M+K]+',
  '[M+3H]+',
  '[M+2H+Na]+',
  '[M+H+2Na]+',
  '[M+3Na]+',
  '[M+H+NH4]+',
  '[M+H+Na]+',
  '[M+H+K]+',
  '[M+ACN+2H]+',
  '[M+2Na]+',
  '[M+2ACN+2H]+',
  '[M+3ACN+2H]+',
  '[M+CH3OH+H]+',
  '[M+ACN+H]+',
  '[M+2Na-H]+',
  '[M+IsoProp+H]+',
  '[M+ACN+Na]+',
  '[M+2K+H]+',
  '[M+DMSO+H]+',
  '[M+2ACN+H]+',
  '[M+IsoProp+Na+H]+',
  '[2M+H]+',
  '[2M+NH4]+',
  '[2M+Na]+',
  '[2M+3H2O+2H]+',
  '[2M+K]+',
  '[2M+ACN+H]+',
  '[2M+ACN+Na]+'],
 ['[M-H]-',
  '[M+CH3COO]-',
  '[M-2H]-',
  '[2M-H]-',
  '[M+H2O-2H]-',
  '[M+COO]-',
  '[M-3H]-',
  '[M+Na-2H]-',
  '[M+Cl]-',
  '[M+K-2H]-',
  '[M+Br]-',
  '[M+CF3COO]-',
  '[2M+COO]-',
  '[2M+CH3COO]-',
  '[3M-H]-']]

### Example 6b - Get Positive Adduct Names

In [117]:
url = BASE_URL + 'get-adduct-names/positive'
response = requests.get(url, verify=False)
json.loads(response.content)

C:\Users\jonst\Anaconda3\envs\MINE\lib\site-packages\urllib3\connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minedatabase.ci.northwestern.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


['[M]+',
 '[M+H]+',
 '[M+NH4]+',
 '[M+Na]+',
 '[M+2H]+',
 '[M+K]+',
 '[M+3H]+',
 '[M+2H+Na]+',
 '[M+H+2Na]+',
 '[M+3Na]+',
 '[M+H+NH4]+',
 '[M+H+Na]+',
 '[M+H+K]+',
 '[M+ACN+2H]+',
 '[M+2Na]+',
 '[M+2ACN+2H]+',
 '[M+3ACN+2H]+',
 '[M+CH3OH+H]+',
 '[M+ACN+H]+',
 '[M+2Na-H]+',
 '[M+IsoProp+H]+',
 '[M+ACN+Na]+',
 '[M+2K+H]+',
 '[M+DMSO+H]+',
 '[M+2ACN+H]+',
 '[M+IsoProp+Na+H]+',
 '[2M+H]+',
 '[2M+NH4]+',
 '[2M+Na]+',
 '[2M+3H2O+2H]+',
 '[2M+K]+',
 '[2M+ACN+H]+',
 '[2M+ACN+Na]+']